In [19]:
import sqlite3 
conn = sqlite3.connect("/home/lsi8505/miraeasset/crawling/opinion_outside.db")
cur = conn.cursor()

import re
from bs4 import BeautifulSoup
import requests
import itertools
from selenium.webdriver.support.ui import Select
import time
import sys
import os
import numpy as np
import pandas as pd
from datetime import datetime, timedelta
import json
client_id = "l5s8tg1g0w" # 개발자센터에서 발급받은 Client ID 값
client_secret = "RPNXQKCp8erMRW4KvKWsME5m3F9QOYVJTZ8Lvj56" # 개발자센터에서 발급받은 Client Secret 값


def get_sentiment(content):
    printout = True
    url="https://naveropenapi.apigw.ntruss.com/sentiment-analysis/v1/analyze"

    headers = {
        "X-NCP-APIGW-API-KEY-ID": client_id,
        "X-NCP-APIGW-API-KEY": client_secret,
        "Content-Type": "application/json"
    }

    data = {
      "content": content
    }

    response = requests.post(url, data=json.dumps(data), headers=headers)
    rescode = response.status_code

    if(rescode == 200):        
        sentiment = json.loads(response.text)['document']['sentiment'] 
        pos_score = json.loads(response.text)['document']['confidence']['positive']
        neg_score = json.loads(response.text)['document']['confidence']['negative']
        neu_score = json.loads(response.text)['document']['confidence']['neutral']
#         if printout:
#             print("\n=== 감성분석 ===")
#             print("판정: {}\n점수: 긍정 {}, 중립 {}, 부정 {}".format(sentiment, pos_score, neu_score, neg_score)) 
#         return sentiment, pos_score, neu_score, neg_score
        return pos_score

    else:
        print("Error : " + response.text)
        return np.nan     
    
    
df_already= pd.read_sql("SELECT * FROM TABLE_BUZZ where SOURCE  = '네이버_종목토론실'",conn)
df_already



df_ttl = pd.DataFrame()

board_nm = 0

while True:
    board_nm +=1
    item_board_url = "https://finance.naver.com/item/board.naver?code=006800&page=" + str(board_nm)
    #         print(item_board_url)
    res = requests.get(item_board_url, headers = {'User-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/96.0.4664.110 Safari/537.36'})#, params=params)
    soup = BeautifulSoup(res.text, "html5lib")
    #         tab_title = soup.findAll('td', attrs={'class':'title'})   
    tab_date = soup.findAll('span', attrs={'class':'tah p10 gray03'})  
    tab_date_txt = [i.text for i in tab_date]

    tab_title = soup.findAll('td', attrs={'class':'title'})  
    tab_title_txt = [i.text for i in tab_title]

    tab_writer = soup.findAll('td', attrs={'class':'p11'})  
    tab_writer_txt = [i.text for i in tab_writer]
    tab_writer_txt = tab_writer_txt[:len(tab_title_txt)]

    url_l = []
    for n in soup.findAll('td', attrs={'class':'title'})  :
        url_l.append("https://finance.naver.com/" +n.findAll('a')[0]['href'])
    url_l
    
    like_l = []
    for s in soup.findAll('tr'): 
        if len(s.findAll('strong')) > 0:
            like_l.append(s.findAll('strong')[0].text)
    like_l[:20]


    df_ = pd.DataFrame()
    df_['DATETIME'] = pd.Series(tab_date_txt).iloc[::2].reset_index(drop=True)
    df_['DATETIME'] =df_['DATETIME'].str.replace(".","").str.replace(" ","-").str.replace(":","")+"00"
    df_['VIEWS'] = pd.Series(tab_date_txt).iloc[1::2].reset_index(drop=True)
    df_['LIKES'] = like_l[:20]
    df_['TITLE'] =  pd.Series(tab_title_txt).str.replace('\n','').str.replace('\t','').reset_index(drop=True)
    df_['CONTENT'] = np.nan
    df_['WRITER'] = pd.Series(tab_writer_txt).str.replace('\n','').str.replace('\t','').reset_index(drop=True)

    df_['SOURCE'] = "네이버_종목토론실"
    df_['RELATED_ITEM'] = "미래에셋증권"
    df_['POSITIVITY'] = np.nan
    df_['OTHERS'] = df_[['WRITER']].to_dict('records')
    df_['OTHERS'] = df_['OTHERS'].astype(str)
    df_['NUM_REPLY'] = df_['TITLE'].apply(lambda x: x.split('[')[1].replace("]","") if len(x.split('[')) == 2 else 0)
    df_['CRAWLING_DATETIME'] = (datetime.now() + timedelta(hours=9)).strftime('%Y%m%d-%H%M%S')
    df_['URL'] =  url_l
    df_


    for i in range(len(df_)):
        df_.loc[i,"POSITIVITY"] = get_sentiment(df_.loc[i,"TITLE"])
    

    df_ = df_[['SOURCE','RELATED_ITEM',"DATETIME","TITLE","CONTENT","POSITIVITY","NUM_REPLY",'URL',"CRAWLING_DATETIME","OTHERS"]]
    df_
    df_ttl = df_ttl.append(df_)
    if float(df_already['DATETIME'].max().replace("-","")) > float(df_ttl['DATETIME'].min().replace("-","")):
        break     
#     print(board_nm)
    



df_ttl = df_ttl[df_ttl['DATETIME'].str.replace("-","").astype(float) > float(df_already['DATETIME'].max().replace("-",""))]
df_ttl

if len(df_ttl) >0:
    df_ttl = df_ttl.reset_index(drop=True)

    df_ttl
    df_ttl.to_sql('TABLE_BUZZ',conn,if_exists = 'append',index = False)
    print((datetime.now() + timedelta(hours=9)).strftime('%Y%m%d-%H%M%S') , " SUCCESS  : ",len(df_ttl))
else:
    print((datetime.now() + timedelta(hours=9)).strftime('%Y%m%d-%H%M%S'), " NO DATA TO UPDATE")

/tmp/ipykernel_79551/3137382348.py:94: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df_['DATETIME'] =df_['DATETIME'].str.replace(".","").str.replace(" ","-").str.replace(":","")+"00"


20220831-105854  NO DATA TO UPDATE


/tmp/ipykernel_79551/3137382348.py:118: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_ttl = df_ttl.append(df_)


In [9]:
import sqlite3 
conn = sqlite3.connect("/home/lsi8505/miraeasset/crawling/opinion_outside.db")
cur = conn.cursor()

import re
from bs4 import BeautifulSoup
import requests
import itertools
from selenium.webdriver.support.ui import Select
import time
import sys
import os
import numpy as np
import pandas as pd
from datetime import datetime, timedelta
import json
client_id = "l5s8tg1g0w" # 개발자센터에서 발급받은 Client ID 값
client_secret = "RPNXQKCp8erMRW4KvKWsME5m3F9QOYVJTZ8Lvj56" # 개발자센터에서 발급받은 Client Secret 값


def get_sentiment(content):
    printout = True
    url="https://naveropenapi.apigw.ntruss.com/sentiment-analysis/v1/analyze"

    headers = {
        "X-NCP-APIGW-API-KEY-ID": client_id,
        "X-NCP-APIGW-API-KEY": client_secret,
        "Content-Type": "application/json"
    }

    data = {
      "content": content
    }

    response = requests.post(url, data=json.dumps(data), headers=headers)
    rescode = response.status_code

    if(rescode == 200):        
        sentiment = json.loads(response.text)['document']['sentiment'] 
        pos_score = json.loads(response.text)['document']['confidence']['positive']
        neg_score = json.loads(response.text)['document']['confidence']['negative']
        neu_score = json.loads(response.text)['document']['confidence']['neutral']
#         if printout:
#             print("\n=== 감성분석 ===")
#             print("판정: {}\n점수: 긍정 {}, 중립 {}, 부정 {}".format(sentiment, pos_score, neu_score, neg_score)) 
#         return sentiment, pos_score, neu_score, neg_score
        return pos_score

    else:
        print("Error : " + response.text)
        return np.nan     
    
    
df_already= pd.read_sql("SELECT * FROM TABLE_BUZZ where SOURCE  = '네이버_종목토론실'",conn)
df_already



df_ttl = pd.DataFrame()

board_nm = 0

while True:
    board_nm +=1
    item_board_url = "https://finance.naver.com/item/board.naver?code=006800&page=" + str(board_nm)
    #         print(item_board_url)
    res = requests.get(item_board_url, headers = {'User-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/96.0.4664.110 Safari/537.36'})#, params=params)
    soup = BeautifulSoup(res.text, "html5lib")
    #         tab_title = soup.findAll('td', attrs={'class':'title'})   
    tab_date = soup.findAll('span', attrs={'class':'tah p10 gray03'})  
    tab_date_txt = [i.text for i in tab_date]

    tab_title = soup.findAll('td', attrs={'class':'title'})  
    tab_title_txt = [i.text for i in tab_title]

    tab_writer = soup.findAll('td', attrs={'class':'p11'})  
    tab_writer_txt = [i.text for i in tab_writer]
    tab_writer_txt = tab_writer_txt[:len(tab_title_txt)]

    url_l = []
    for n in soup.findAll('td', attrs={'class':'title'})  :
        url_l.append("https://finance.naver.com/" +n.findAll('a')[0]['href'])
    url_l
    
    like_l = []
    for s in soup.findAll('tr'): 
        if len(s.findAll('strong')) > 0:
            like_l.append(s.findAll('strong')[0].text)
    like_l[:20]


    df_ = pd.DataFrame()
    df_['DATETIME'] = pd.Series(tab_date_txt).iloc[::2].reset_index(drop=True)
    df_['DATETIME'] =df_['DATETIME'].str.replace(".","").str.replace(" ","-").str.replace(":","")+"00"
    df_['VIEWS'] = pd.Series(tab_date_txt).iloc[1::2].reset_index(drop=True)
    df_['LIKES'] = like_l[:20]
    df_['TITLE'] =  pd.Series(tab_title_txt).str.replace('\n','').str.replace('\t','').reset_index(drop=True)
    df_['CONTENT'] = np.nan
    df_['WRITER'] = pd.Series(tab_writer_txt).str.replace('\n','').str.replace('\t','').reset_index(drop=True)

    df_['SOURCE'] = "네이버_종목토론실"
    df_['RELATED_ITEM'] = "미래에셋증권"
    df_['POSITIVITY'] = np.nan
    df_['OTHERS'] = df_[['WRITER']].to_dict('records')
    df_['OTHERS'] = df_['OTHERS'].astype(str)
    df_['NUM_REPLY'] = df_['TITLE'].apply(lambda x: x.split('[')[1].replace("]","") if len(x.split('[')) == 2 else 0)
    df_['CRAWLING_DATETIME'] = (datetime.now() + timedelta(hours=9)).strftime('%Y%m%d-%H%M%S')
    df_['URL'] =  url_l
    df_


    for i in range(len(df_)):
        df_.loc[i,"POSITIVITY"] = get_sentiment(df_.loc[i,"TITLE"])
    

    df_ = df_[['SOURCE','RELATED_ITEM',"DATETIME","TITLE","CONTENT","POSITIVITY","NUM_REPLY",'URL',"CRAWLING_DATETIME","OTHERS"]]
    df_
    df_ttl = df_ttl.append(df_)
    if float(df_already['DATETIME'].max().replace("-","")) > float(df_ttl['DATETIME'].min().replace("-","")):
        break        
#     print(board_nm)
    


/tmp/ipykernel_79551/1614280575.py:94: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df_['DATETIME'] =df_['DATETIME'].str.replace(".","").str.replace(" ","-").str.replace(":","")+"00"
/tmp/ipykernel_79551/1614280575.py:118: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_ttl = df_ttl.append(df_)


In [10]:
df_ttl = df_ttl[df_ttl['DATETIME'].str.replace("-","").astype(float) > float(df_already['DATETIME'].max().replace("-",""))]
df_ttl

,SOURCE,RELATED_ITEM,DATETIME,TITLE,CONTENT,POSITIVITY,NUM_REPLY,URL,CRAWLING_DATETIME,OTHERS
0,네이버_종목토론실,미래에셋증권,20220831-100500,키움증권 급등중,NaN,99.907550,0,https://finance.naver.com//item/board_read.nav...,20220831-105541,{'WRITER': 'pksk****'}
1,네이버_종목토론실,미래에셋증권,20220831-095500,이평선 모이는중 조만간 단기 급등,NaN,99.735820,0,https://finance.naver.com//item/board_read.nav...,20220831-105541,{'WRITER': 'pksk****'}
2,네이버_종목토론실,미래에셋증권,20220831-095100,"JP모간16,847골드만삭스8,474[1]",NaN,0.019560,1,https://finance.naver.com//item/board_read.nav...,20220831-105541,{'WRITER': 'hori****'}
3,네이버_종목토론실,미래에셋증권,20220831-094000,개미 다 털린거 같은데? 조만간 급등할듯,NaN,49.946503,0,https://finance.naver.com//item/board_read.nav...,20220831-105541,{'WRITER': 'pksk****'}
4,네이버_종목토론실,미래에셋증권,20220831-080400,"개인-214,478[1]",NaN,0.018545,1,https://finance.naver.com//item/board_read.nav...,20220831-105541,{'WRITER': 'hori****'}
5,네이버_종목토론실,미래에셋증권,20220830-212700,mts ㅅㅆㄹㄱ[1],NaN,0.018479,1,https://finance.naver.com//item/board_read.nav...,20220831-105541,{'WRITER': 'cnyy****'}
6,네이버_종목토론실,미래에셋증권,20220830-165400,ㅋㅋㅋ 지루한 횡보,NaN,0.138840,0,https://finance.naver.com//item/board_read.nav...,20220831-105541,{'WRITER': 'suin****'}
7,네이버_종목토론실,미래에셋증권,20220830-151900,와 너 올라가는구나,NaN,0.445714,0,https://finance.naver.com//item/board_read.nav...,20220831-105541,{'WRITER': 'ying****'}
8,네이버_종목토론실,미래에셋증권,20220830-150600,6600원,NaN,0.026288,0,https://finance.naver.com//item/board_read.nav...,20220831-105541,{'WRITER': 'deck****'}
9,네이버_종목토론실,미래에셋증권,20220830-122500,언제 7000원올까요?,NaN,0.036618,0,https://finance.naver.com//item/board_read.nav...,20220831-105541,{'WRITER': 'dhgu****'}


In [12]:

if len(df_ttl) >0:
    df_ttl = df_ttl.reset_index(drop=True)

    df_ttl
    df_ttl.to_sql('TABLE_BUZZ',conn,if_exists = 'append',index = False)
    print((datetime.now() + timedelta(hours=9)).strftime('%Y%m%d-%H%M%S') , " SUCCESS  : ",len(df_ttl))
else:
    print((datetime.now() + timedelta(hours=9)).strftime('%Y%m%d-%H%M%S'), " NO DATA TO UPDATE")




20220831-105614  SUCCESS  :  12


In [2]:
import sqlite3 
conn = sqlite3.connect("/home/porter/miraeasset/crawling/opinion_outside.db")
cur = conn.cursor()

In [3]:
import re
from bs4 import BeautifulSoup
import requests
import itertools
from selenium.webdriver.support.ui import Select
import time
import sys
import os
import numpy as np
import pandas as pd
from datetime import datetime
import json
client_id = "l5s8tg1g0w" # 개발자센터에서 발급받은 Client ID 값
client_secret = "RPNXQKCp8erMRW4KvKWsME5m3F9QOYVJTZ8Lvj56" # 개발자센터에서 발급받은 Client Secret 값


def get_sentiment(content):
    printout = True
    url="https://naveropenapi.apigw.ntruss.com/sentiment-analysis/v1/analyze"

    headers = {
        "X-NCP-APIGW-API-KEY-ID": client_id,
        "X-NCP-APIGW-API-KEY": client_secret,
        "Content-Type": "application/json"
    }

    data = {
      "content": content
    }

    response = requests.post(url, data=json.dumps(data), headers=headers)
    rescode = response.status_code

    if(rescode == 200):        
        sentiment = json.loads(response.text)['document']['sentiment'] 
        pos_score = json.loads(response.text)['document']['confidence']['positive']
        neg_score = json.loads(response.text)['document']['confidence']['negative']
        neu_score = json.loads(response.text)['document']['confidence']['neutral']
#         if printout:
#             print("\n=== 감성분석 ===")
#             print("판정: {}\n점수: 긍정 {}, 중립 {}, 부정 {}".format(sentiment, pos_score, neu_score, neg_score)) 
#         return sentiment, pos_score, neu_score, neg_score
        return pos_score

    else:
        print("Error : " + response.text)
        return np.nan     

In [45]:
df_already= pd.read_sql("SELECT * FROM TABLE_BUZZ where SOURCE  = '네이버_종목토론실' order by CRAWLING_DATETIME desc",conn)
df_already.head(10)

,SOURCE,RELATED_ITEM,DATETIME,TITLE,CONTENT,POSITIVITY,NUM_REPLY,URL,CRAWLING_DATETIME,OTHERS
0,네이버_종목토론실,미래에셋증권,20220816-161600,반가워요,None,2.626338,0,https://finance.naver.com//item/board_read.nav...,20220819-170225,{'WRITER': 'dltk****'}
1,네이버_종목토론실,미래에셋증권,20220816-145400,올라갈 생각이 없구만!,None,0.007683,0,https://finance.naver.com//item/board_read.nav...,20220819-170225,{'WRITER': 'ban8****'}
2,네이버_종목토론실,미래에셋증권,20220816-140400,종가,None,0.030199,0,https://finance.naver.com//item/board_read.nav...,20220819-170225,{'WRITER': 'suin****'}
3,네이버_종목토론실,미래에셋증권,20220816-130600,주식창이 아주 그냥 미쳐가네,None,0.005338,0,https://finance.naver.com//item/board_read.nav...,20220819-170225,{'WRITER': 'tkdg****'}
4,네이버_종목토론실,미래에셋증권,20220816-161600,반가워요,None,2.626338,0,https://finance.naver.com//item/board_read.nav...,20220819-170225,{'WRITER': 'dltk****'}
5,네이버_종목토론실,미래에셋증권,20220816-145400,올라갈 생각이 없구만!,None,0.007683,0,https://finance.naver.com//item/board_read.nav...,20220819-170225,{'WRITER': 'ban8****'}
6,네이버_종목토론실,미래에셋증권,20220816-140400,종가,None,0.030199,0,https://finance.naver.com//item/board_read.nav...,20220819-170225,{'WRITER': 'suin****'}
7,네이버_종목토론실,미래에셋증권,20220816-130600,주식창이 아주 그냥 미쳐가네,None,0.005338,0,https://finance.naver.com//item/board_read.nav...,20220819-170225,{'WRITER': 'tkdg****'}
8,네이버_종목토론실,미래에셋증권,20220818-133900,미래에셋증권[2],None,0.021002,2,https://finance.naver.com//item/board_read.nav...,20220819-170222,{'WRITER': 'rlaa****'}
9,네이버_종목토론실,미래에셋증권,20220818-125700,네이버에 LIKEBEE ~~~,None,0.001436,0,https://finance.naver.com//item/board_read.nav...,20220819-170222,{'WRITER': 'js10****'}


In [35]:
df_ttl = pd.DataFrame()

board_nm = 0

while True:
    board_nm +=1
    item_board_url = "https://finance.naver.com/item/board.naver?code=006800&page=" + str(board_nm)
    #         print(item_board_url)
    res = requests.get(item_board_url, headers = {'User-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/96.0.4664.110 Safari/537.36'})#, params=params)
    soup = BeautifulSoup(res.text, "html5lib")
    #         tab_title = soup.findAll('td', attrs={'class':'title'})   
    tab_date = soup.findAll('span', attrs={'class':'tah p10 gray03'})  
    tab_date_txt = [i.text for i in tab_date]

    tab_title = soup.findAll('td', attrs={'class':'title'})  
    tab_title_txt = [i.text for i in tab_title]

    tab_writer = soup.findAll('td', attrs={'class':'p11'})  
    tab_writer_txt = [i.text for i in tab_writer]
    tab_writer_txt = tab_writer_txt[:len(tab_title_txt)]

    url_l = []
    for n in soup.findAll('td', attrs={'class':'title'})  :
        url_l.append("https://finance.naver.com/" +n.findAll('a')[0]['href'])
    url_l
    
    like_l = []
    for s in soup.findAll('tr'): 
        if len(s.findAll('strong')) > 0:
            like_l.append(s.findAll('strong')[0].text)
    like_l[:20]


    df_ = pd.DataFrame()
    df_['DATETIME'] = pd.Series(tab_date_txt).iloc[::2].reset_index(drop=True)
    df_['DATETIME'] =df_['DATETIME'].str.replace(".","").str.replace(" ","-").str.replace(":","")+"00"
    df_['VIEWS'] = pd.Series(tab_date_txt).iloc[1::2].reset_index(drop=True)
    df_['LIKES'] = like_l[:20]
    df_['TITLE'] =  pd.Series(tab_title_txt).str.replace('\n','').str.replace('\t','').reset_index(drop=True)
    df_['CONTENT'] = np.nan
    df_['WRITER'] = pd.Series(tab_writer_txt).str.replace('\n','').str.replace('\t','').reset_index(drop=True)

    df_['SOURCE'] = "네이버_종목토론실"
    df_['RELATED_ITEM'] = "미래에셋증권"
    df_['POSITIVITY'] = np.nan
    df_['OTHERS'] = df_[['WRITER']].to_dict('records')
    df_['OTHERS'] = df_['OTHERS'].astype(str)
    df_['NUM_REPLY'] = df_['TITLE'].apply(lambda x: x.split('[')[1].replace("]","") if len(x.split('[')) == 2 else 0)
    df_['CRAWLING_DATETIME'] = datetime.now().strftime('%Y%m%d-%H%M%S')
    df_['URL'] =  url_l
    df_


    for i in range(len(df_)):
        df_.loc[i,"POSITIVITY"] = get_sentiment(df_.loc[i,"TITLE"])
    

    df_ = df_[['SOURCE','RELATED_ITEM',"DATETIME","TITLE","CONTENT","POSITIVITY","NUM_REPLY",'URL',"CRAWLING_DATETIME","OTHERS"]]
    df_
    df_ttl = df_ttl.append(df_)
    if len(df_[df_['DATETIME']==df_already['DATETIME'].max()]) >0:
        break
        
    print(board_nm)
df_ttl = df_ttl.reset_index(drop=True)
df_ttl = df_ttl.iloc[:df_ttl[df_ttl['DATETIME']==df_already['DATETIME'].max()].index[0]]
df_ttl

/tmp/ipykernel_109994/252012253.py:36: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df_['DATETIME'] =df_['DATETIME'].str.replace(".","").str.replace(" ","-").str.replace(":","")+"00"


1
2


,SOURCE,RELATED_ITEM,DATETIME,TITLE,CONTENT,POSITIVITY,NUM_REPLY,URL,CRAWLING_DATETIME,OTHERS
0,네이버_종목토론실,미래에셋증권,20220819-164000,박현주 제재좀 하자 개인회사 너무 이용한...,NaN,0.005494,0,https://finance.naver.com//item/board_read.nav...,20220819-170155,{'WRITER': 'amgo****'}
1,네이버_종목토론실,미래에셋증권,20220819-160600,미래에셋 주가는...,NaN,0.025233,0,https://finance.naver.com//item/board_read.nav...,20220819-170155,{'WRITER': 'wlsd****'}
2,네이버_종목토론실,미래에셋증권,20220819-153100,미래에셋증권,NaN,0.019888,0,https://finance.naver.com//item/board_read.nav...,20220819-170155,{'WRITER': 'mhjc****'}
3,네이버_종목토론실,미래에셋증권,20220819-152400,미래에셋 -> 타증권사 이동,NaN,0.021194,0,https://finance.naver.com//item/board_read.nav...,20220819-170155,{'WRITER': 'focu****'}
4,네이버_종목토론실,미래에셋증권,20220819-131700,미래에셋은 고객 신경 쓰긴 쓰나요?[1],NaN,0.087408,1,https://finance.naver.com//item/board_read.nav...,20220819-170155,{'WRITER': 'pksk****'}
5,네이버_종목토론실,미래에셋증권,20220819-110900,CEO 연봉[1],NaN,0.019375,1,https://finance.naver.com//item/board_read.nav...,20220819-170155,{'WRITER': 'heon****'}
6,네이버_종목토론실,미래에셋증권,20220819-112100,그 정도는 받아야제[1],NaN,0.110950,1,https://finance.naver.com//item/board_read.nav...,20220819-170155,{'WRITER': 'foxm****'}
7,네이버_종목토론실,미래에셋증권,20220819-105900,6억6800만원 아파트를 산다고 가정하면[2],NaN,0.016766,2,https://finance.naver.com//item/board_read.nav...,20220819-170155,{'WRITER': 'foxm****'}
8,네이버_종목토론실,미래에셋증권,20220819-102200,미래에셋증권은 주당자산가치가,NaN,0.043348,0,https://finance.naver.com//item/board_read.nav...,20220819-170155,{'WRITER': 'foxm****'}
9,네이버_종목토론실,미래에셋증권,20220819-100600,금일 예상 종가 6780원[2],NaN,0.029293,2,https://finance.naver.com//item/board_read.nav...,20220819-170155,{'WRITER': 'foxm****'}


In [44]:
df_ttl.to_sql('TABLE_BUZZ',conn,if_exists = 'append',index = False)
df_ttl.tail(10)

,SOURCE,RELATED_ITEM,DATETIME,TITLE,CONTENT,POSITIVITY,NUM_REPLY,URL,CRAWLING_DATETIME,OTHERS
34,네이버_종목토론실,미래에셋증권,20220817-094700,자사주 그리고 공매도,NaN,0.035148,0,https://finance.naver.com//item/board_read.nav...,20220819-170222,{'WRITER': 'suin****'}
35,네이버_종목토론실,미래에셋증권,20220817-010900,미래에셋증권,NaN,0.019888,0,https://finance.naver.com//item/board_read.nav...,20220819-170222,{'WRITER': 'pksk****'}
36,네이버_종목토론실,미래에셋증권,20220816-201000,구버젼 앱 까는 방법 아시는분요..[1],NaN,0.030117,1,https://finance.naver.com//item/board_read.nav...,20220819-170222,{'WRITER': 'kkwa****'}
37,네이버_종목토론실,미래에셋증권,20220816-195800,미래에셋 고객들 이탈 시작되었어요[1],NaN,0.027032,1,https://finance.naver.com//item/board_read.nav...,20220819-170222,{'WRITER': 'kkwa****'}
38,네이버_종목토론실,미래에셋증권,20220816-175900,미래에셋 -> 타증권사[1],NaN,0.018887,1,https://finance.naver.com//item/board_read.nav...,20220819-170222,{'WRITER': 'focu****'}
39,네이버_종목토론실,미래에셋증권,20220816-171500,앱 쫌 원상태로 돌려놔줘라,NaN,0.003479,0,https://finance.naver.com//item/board_read.nav...,20220819-170222,{'WRITER': 'ghba****'}
40,네이버_종목토론실,미래에셋증권,20220816-161600,반가워요,NaN,2.626338,0,https://finance.naver.com//item/board_read.nav...,20220819-170225,{'WRITER': 'dltk****'}
41,네이버_종목토론실,미래에셋증권,20220816-145400,올라갈 생각이 없구만!,NaN,0.007683,0,https://finance.naver.com//item/board_read.nav...,20220819-170225,{'WRITER': 'ban8****'}
42,네이버_종목토론실,미래에셋증권,20220816-140400,종가,NaN,0.030199,0,https://finance.naver.com//item/board_read.nav...,20220819-170225,{'WRITER': 'suin****'}
43,네이버_종목토론실,미래에셋증권,20220816-130600,주식창이 아주 그냥 미쳐가네,NaN,0.005338,0,https://finance.naver.com//item/board_read.nav...,20220819-170225,{'WRITER': 'tkdg****'}


In [7]:
import sqlite3 
conn = sqlite3.connect("/home/porter/miraeasset/crawling/opinion_outside.db")
cur = conn.cursor()

In [1]:
import re
from bs4 import BeautifulSoup
import requests
import itertools
from selenium.webdriver.support.ui import Select
import time
import sys
import os
import numpy as np
import pandas as pd
from datetime import datetime
import json
client_id = "l5s8tg1g0w" # 개발자센터에서 발급받은 Client ID 값
client_secret = "RPNXQKCp8erMRW4KvKWsME5m3F9QOYVJTZ8Lvj56" # 개발자센터에서 발급받은 Client Secret 값


def get_sentiment(content):
    printout = True
    url="https://naveropenapi.apigw.ntruss.com/sentiment-analysis/v1/analyze"

    headers = {
        "X-NCP-APIGW-API-KEY-ID": client_id,
        "X-NCP-APIGW-API-KEY": client_secret,
        "Content-Type": "application/json"
    }

    data = {
      "content": content
    }

    response = requests.post(url, data=json.dumps(data), headers=headers)
    rescode = response.status_code

    if(rescode == 200):        
        sentiment = json.loads(response.text)['document']['sentiment'] 
        pos_score = json.loads(response.text)['document']['confidence']['positive']
        neg_score = json.loads(response.text)['document']['confidence']['negative']
        neu_score = json.loads(response.text)['document']['confidence']['neutral']
#         if printout:
#             print("\n=== 감성분석 ===")
#             print("판정: {}\n점수: 긍정 {}, 중립 {}, 부정 {}".format(sentiment, pos_score, neu_score, neg_score)) 
#         return sentiment, pos_score, neu_score, neg_score
        return pos_score

    else:
        print("Error : " + response.text)
        return np.nan     

In [2]:
df_ttl = pd.DataFrame()

board_nm = 1

while True:
    board_nm +=1
    item_board_url = "https://finance.naver.com/item/board.naver?code=006800&page=" + str(board_nm)
    #         print(item_board_url)
    res = requests.get(item_board_url, headers = {'User-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/96.0.4664.110 Safari/537.36'})#, params=params)
    soup = BeautifulSoup(res.text, "html5lib")
    #         tab_title = soup.findAll('td', attrs={'class':'title'})   
    tab_date = soup.findAll('span', attrs={'class':'tah p10 gray03'})  
    tab_date_txt = [i.text for i in tab_date]

    tab_title = soup.findAll('td', attrs={'class':'title'})  
    tab_title_txt = [i.text for i in tab_title]

    tab_writer = soup.findAll('td', attrs={'class':'p11'})  
    tab_writer_txt = [i.text for i in tab_writer]
    tab_writer_txt = tab_writer_txt[:len(tab_title_txt)]

    url_l = []
    for n in soup.findAll('td', attrs={'class':'title'})  :
        url_l.append("https://finance.naver.com/" +n.findAll('a')[0]['href'])
    url_l
    
    like_l = []
    for s in soup.findAll('tr'): 
        if len(s.findAll('strong')) > 0:
            like_l.append(s.findAll('strong')[0].text)
    like_l[:20]


    df_ = pd.DataFrame()
    df_['DATETIME'] = pd.Series(tab_date_txt).iloc[::2].reset_index(drop=True)
    df_['DATETIME'] =df_['DATETIME'].str.replace(".","").str.replace(" ","-").str.replace(":","")+"00"
    df_['VIEWS'] = pd.Series(tab_date_txt).iloc[1::2].reset_index(drop=True)
    df_['LIKES'] = like_l[:20]
    df_['TITLE'] =  pd.Series(tab_title_txt).str.replace('\n','').str.replace('\t','').reset_index(drop=True)
    df_['CONTENT'] = np.nan
    df_['WRITER'] = pd.Series(tab_writer_txt).str.replace('\n','').str.replace('\t','').reset_index(drop=True)

    df_['SOURCE'] = "네이버_종목토론실"
    df_['RELATED_ITEM'] = "미래에셋증권"
    df_['POSITIVITY'] = np.nan
    df_['OTHERS'] = df_[['WRITER']].to_dict('records')
    df_['NUM_REPLY'] = df_['TITLE'].apply(lambda x: x.split('[')[1].replace("]","") if len(x.split('[')) == 2 else 0)
    df_['CRAWLING_DATETIME'] = datetime.now().strftime('%Y%m%d-%H%M%S')
    df_['URL'] =  url_l
    df_


    for i in range(len(df_)):
        df_.loc[i,"POSITIVITY"] = get_sentiment(df_.loc[i,"TITLE"])
    

    df_ = df_[['SOURCE','RELATED_ITEM',"DATETIME","TITLE","CONTENT","POSITIVITY","NUM_REPLY",'URL',"CRAWLING_DATETIME","OTHERS"]]
    df_
    df_ttl = df_ttl.append(df_)
    if df_['DATETIME'].str[:4].min() == '2021':
        break
    print(board_nm)

/tmp/ipykernel_40007/2638858422.py:36: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df_['DATETIME'] =df_['DATETIME'].str.replace(".","").str.replace(" ","-").str.replace(":","")+"00"


2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100


In [8]:
df_ttl.to_csv("naver_jongto.csv",encoding='utf-16')
df_ttl

,SOURCE,RELATED_ITEM,DATETIME,TITLE,CONTENT,POSITIVITY,NUM_REPLY,URL,CRAWLING_DATETIME,OTHERS
0,네이버_종목토론실,미래에셋증권,20220816-125200,고생 많으시네,NaN,99.635650,0,https://finance.naver.com//item/board_read.nav...,20220817-143541,{'WRITER': 'odgy****'}
1,네이버_종목토론실,미래에셋증권,20220816-121100,환장하네,NaN,0.005247,0,https://finance.naver.com//item/board_read.nav...,20220817-143541,{'WRITER': 'kyd6****'}
2,네이버_종목토론실,미래에셋증권,20220816-115900,주문 못하겠다 미 친눔들아,NaN,0.034474,0,https://finance.naver.com//item/board_read.nav...,20220817-143541,{'WRITER': 'kyd6****'}
3,네이버_종목토론실,미래에셋증권,20220816-115500,개 더러운 주식...,NaN,0.008190,0,https://finance.naver.com//item/board_read.nav...,20220817-143541,{'WRITER': 'pdy1****'}
4,네이버_종목토론실,미래에셋증권,20220816-114000,옛날 성국이형 있을때가 좋았다[1],NaN,85.176476,1,https://finance.naver.com//item/board_read.nav...,20220817-143541,{'WRITER': 'seri****'}
...,...,...,...,...,...,...,...,...,...,...
15,네이버_종목토론실,미래에셋증권,20220101-090900,올해는 개행주 현만이의 역대급저주로,NaN,0.009513,0,https://finance.naver.com//item/board_read.nav...,20220817-144647,{'WRITER': 'syju****'}
16,네이버_종목토론실,미래에셋증권,20211231-102600,배당금 얼마 주나요?[6],NaN,0.020373,6,https://finance.naver.com//item/board_read.nav...,20220817-144647,{'WRITER': 'emai****'}
17,네이버_종목토론실,미래에셋증권,20211230-223000,왜 출금이 안되지.미래에셋증권 쓰는데[1],NaN,0.005959,1,https://finance.naver.com//item/board_read.nav...,20220817-144647,{'WRITER': 'kkwa****'}
18,네이버_종목토론실,미래에셋증권,20211230-211100,개행주 현만이 쫏아내지 않는한,NaN,0.007881,0,https://finance.naver.com//item/board_read.nav...,20220817-144647,{'WRITER': 'syju****'}


In [11]:
df_ttl['OTHERS'] = df_ttl['OTHERS'].astype(str)
df_ttl.to_sql('TABLE_BUZZ',conn,if_exists = 'append',index = False)
df_ttl

,SOURCE,RELATED_ITEM,DATETIME,TITLE,CONTENT,POSITIVITY,NUM_REPLY,URL,CRAWLING_DATETIME,OTHERS
0,네이버_종목토론실,미래에셋증권,20220816-125200,고생 많으시네,NaN,99.635650,0,https://finance.naver.com//item/board_read.nav...,20220817-143541,{'WRITER': 'odgy****'}
1,네이버_종목토론실,미래에셋증권,20220816-121100,환장하네,NaN,0.005247,0,https://finance.naver.com//item/board_read.nav...,20220817-143541,{'WRITER': 'kyd6****'}
2,네이버_종목토론실,미래에셋증권,20220816-115900,주문 못하겠다 미 친눔들아,NaN,0.034474,0,https://finance.naver.com//item/board_read.nav...,20220817-143541,{'WRITER': 'kyd6****'}
3,네이버_종목토론실,미래에셋증권,20220816-115500,개 더러운 주식...,NaN,0.008190,0,https://finance.naver.com//item/board_read.nav...,20220817-143541,{'WRITER': 'pdy1****'}
4,네이버_종목토론실,미래에셋증권,20220816-114000,옛날 성국이형 있을때가 좋았다[1],NaN,85.176476,1,https://finance.naver.com//item/board_read.nav...,20220817-143541,{'WRITER': 'seri****'}
...,...,...,...,...,...,...,...,...,...,...
15,네이버_종목토론실,미래에셋증권,20220101-090900,올해는 개행주 현만이의 역대급저주로,NaN,0.009513,0,https://finance.naver.com//item/board_read.nav...,20220817-144647,{'WRITER': 'syju****'}
16,네이버_종목토론실,미래에셋증권,20211231-102600,배당금 얼마 주나요?[6],NaN,0.020373,6,https://finance.naver.com//item/board_read.nav...,20220817-144647,{'WRITER': 'emai****'}
17,네이버_종목토론실,미래에셋증권,20211230-223000,왜 출금이 안되지.미래에셋증권 쓰는데[1],NaN,0.005959,1,https://finance.naver.com//item/board_read.nav...,20220817-144647,{'WRITER': 'kkwa****'}
18,네이버_종목토론실,미래에셋증권,20211230-211100,개행주 현만이 쫏아내지 않는한,NaN,0.007881,0,https://finance.naver.com//item/board_read.nav...,20220817-144647,{'WRITER': 'syju****'}


In [13]:
df_aa= pd.read_sql('SELECT * FROM TABLE_BUZZ',conn)
df_aa.SOURCE.unique()

array(['네이버_종목토론실', '블라인드_주식투자'], dtype=object)